In [ ]:
# conda install conda-forge::sentence-transformers 5.1.0
# conda install conda-forge::llama-cpp-python 0.2.24

In [6]:
import psycopg2

conn = psycopg2.connect(
    host="rag-data",
    port=5432,
    dbname="appdb",
    user="appuser",
    password="secret"
)

cur = conn.cursor()



In [ ]:
cur.execute("SELECT * FROM documents_e5 LIMIT 2")
cur.fetchall()

psql -h localhost -d appdb -U appuser

In [ ]:
cur.execute("SELECT * FROM documents_ruBert LIMIT 2")
cur.fetchall()

In [ ]:


def chunk_text(text, tokenizer=tokenizer, chunk_size=CHUNK_SIZE, overlap=OVERLAP, max_length=MAX_LENGTH):
    chunk_size = min(chunk_size, max_length)
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start+chunk_size, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        start += chunk_size - overlap
    return chunks

def embed(text: str):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_LENGTH
        )#.to(device)
    with torch.no_grad():
        emb = model(**inputs).last_hidden_state.mean(dim=1)
    return emb[0].cpu().numpy()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

cache_dir="/root/.cache/huggingface" #bad

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-large", cache_dir=cache_dir)
model = AutoModelForMaskedLM.from_pretrained("ai-forever/ruBert-large", cache_dir=cache_dir)

/opt/conda/envs/rag_sys/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ai-forever/ruBert-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
import torch
print("torch:", torch.__version__)
print("torch cuda:", torch.version.cuda)
print("is_available:", torch.cuda.is_available())
print("device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)
print("capability (SM):", torch.cuda.get_device_capability(0) if torch.cuda.is_available() else None)
print("arch list:", torch.cuda.get_arch_list())


torch: 2.8.0+cu128
torch cuda: 12.8
is_available: True
device: NVIDIA GeForce RTX 5070
capability (SM): (12, 0)
arch list: ['sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90', 'sm_100', 'sm_120']


In [ ]:
LLM_MODEL_PATH = "/root/.cache/huggingface/hub/models--ai-forever--ruBert-large/snapshots/efdc76b4678bc5c9a51642a4a5364371a89cea96"

from transformers import AutoTokenizer, AutoModel
import torch

llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_PATH)
llm_model = AutoModel.from_pretrained(
    LLM_MODEL_PATH,
    dtype=torch.float16,
    device_map="cuda"
)

prompt = "Привет, как я могу к тебе обращаться?"
inputs = llm_tokenizer(prompt, return_tensors="pt").to(llm_model.device)
with torch.no_grad():
    output = llm_model.generate(
        **inputs,
        max_new_tokes=512,
        temperature = 0.7,
        do_sample=True
    )
llm_tokenizer.decode(output[0], skip_special_tokens=True)
